In [2]:
#pip install feedparser python-dotenv openai

In [3]:
# AI Knowledge Delivery System 

# AI Knowledge Delivery System – Proof of Concept
# 
# This script implements a fully working prototype of an automated AI-news intelligence pipeline.
# It performs three core tasks end-to-end:
#   1. Collects AI-related news from multiple RSS sources
#   2. Summarises and prioritises the most relevant developments using a Large Language Model
#   3. Sends a daily digest by email via SMTP
#
# The architecture is modular: fetching, summarisation, and delivery are each implemented as 
# discrete functions (tools), allowing seamless migration into a full agent-based workflow.
#
# The long-term design is inspired by multi-agent systems: external-signal collectors, internal 
# context analyzers (via RAG), and an orchestrator that routes the right knowledge to the right team.
#
# This file represents the Phase 1 / Phase 2 implementation:
# - Python automation pipeline
# - Early agent orchestration using LangChain
# - Ready for future expansion into LangGraph multi-agent architecture
#
# Note: Requires an OpenAI API key and SMTP credentials via .env configuration.

In this code, we 
- define the functions (which becomes then the "tools" for our agent) using LLM
- make a homemade orchestrator loop (v1) 
- make a nicer orchestrator using Langchain (v2)

In [4]:
import os
from datetime import datetime, timezone

import feedparser
from dotenv import load_dotenv
from openai import OpenAI

# Load variables from .env file (OPENAI_API_KEY)
load_dotenv()

# Create OpenAI client
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in your .env file")

client = OpenAI(api_key=OPENAI_API_KEY)

print("Setup complete ")


Setup complete 


In [5]:
# A few example AI-related RSS feeds
RSS_FEEDS = [
    "https://openai.com/blog/rss.xml",
    "https://www.theverge.com/rss/ai-artificial-intelligence/index.xml",
    "https://feeds.feedburner.com/Techcrunch/artificial-intelligence",
]

RSS_FEEDS


['https://openai.com/blog/rss.xml',
 'https://www.theverge.com/rss/ai-artificial-intelligence/index.xml',
 'https://feeds.feedburner.com/Techcrunch/artificial-intelligence']

#### Function to fetch articles

In [6]:
def fetch_articles_from_rss(feeds):
    """
    Given a list of RSS feed URLs, return a list of article dicts:
    {title, link, summary, source}
    """
    articles = []

    for url in feeds:
        parsed = feedparser.parse(url)

        # Get the source name (e.g. "OpenAI", "The Verge")
        source_title = parsed.feed.get("title", url)

        # Take the first few entries from each feed
        for entry in parsed.entries[:10]:
            articles.append(
                {
                    "title": entry.get("title", "").strip(),
                    "link": entry.get("link", "").strip(),
                    "summary": entry.get("summary", "").strip(),
                    "source": source_title,
                }
            )
    return articles

# Quick test
articles_test = fetch_articles_from_rss(RSS_FEEDS)
len(articles_test), articles_test[:2]


(20,
 [{'title': 'OpenAI and Foxconn collaborate to strengthen U.S. manufacturing across the AI supply chain',
   'link': 'https://openai.com/index/openai-and-foxconn-collaborate',
   'summary': 'OpenAI and Foxconn are collaborating to design and manufacture next-generation AI infrastructure hardware in the U.S. The partnership will develop multiple generations of data-center systems, strengthen U.S. supply chains, and build key components domestically to accelerate advanced AI infrastructure.',
   'source': 'OpenAI News'},
  {'title': 'Helping 1,000 small businesses build with AI',
   'link': 'https://openai.com/index/small-business-ai-jam',
   'summary': 'OpenAI is partnering with DoorDash, SCORE, and local organizations to help 1,000 small businesses build with AI. The Small Business AI Jam gives Main Street business owners hands-on tools and training to compete and grow.',
   'source': 'OpenAI News'}])

#### Summarise with the LLM

- Takes the list of articles
- Creates a prompt
- Calls the model
- Returns a nice digest as text (Markdown for now)

In [7]:
def make_news_digest_markdown(articles):
    """
    Use OpenAI to create a short AI news digest in Markdown format.
    """
    if not articles:
        return "# Daily AI Briefing\n\nNo AI articles found today."

    # Build a text block with the most important fields
    text_block_lines = []
    for i, a in enumerate(articles[:20], start=1):  # limit to 20 articles
        text_block_lines.append(
            f"[{i}] {a['title']} ({a['source']})\n"
            f"{a['summary']}\n"
            f"Link: {a['link']}\n"
        )

    text_block = "\n".join(text_block_lines)
    today_str = datetime.now(timezone.utc).strftime("%Y-%m-%d")

    user_prompt = f"""
You are creating a short daily AI news digest for a busy reader.

Today is {today_str}.

Here are some AI-related articles:

{text_block}

Please:
- Pick the 3–5 most important stories.
- For each, write a short headline and a 2–3 sentence summary.
- Include exactly one link per story (pick the best one).
- At the end, if there are any remaining interesting articles, 
  add a section "Other notable news" with bullet points (1 sentence + link each).

Write your answer in Markdown.
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",   # you can change to another model if you prefer
        messages=[
            {"role": "system", "content": "You are a concise AI news summariser."},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.3,
    )

    digest_markdown = response.choices[0].message.content
    return digest_markdown

# Quick test with the test articles we fetched before
digest_preview = make_news_digest_markdown(articles_test)
print(digest_preview[:1000])  # show first 1000 characters


# AI Daily News Digest – 2025-11-24

### OpenAI and Foxconn Team Up to Boost U.S. AI Hardware Manufacturing  
OpenAI and Foxconn are partnering to design and produce next-generation AI infrastructure hardware domestically in the U.S. This collaboration aims to strengthen supply chains and accelerate advanced AI data-center systems development.  
[Read more](https://openai.com/index/openai-and-foxconn-collaborate)

### OpenAI Launches Small Business AI Jam to Empower 1,000 Businesses  
OpenAI is collaborating with DoorDash, SCORE, and local groups to provide hands-on AI tools and training to 1,000 small businesses. The initiative helps Main Street entrepreneurs compete and grow using AI technologies.  
[Read more](https://openai.com/index/small-business-ai-jam)

### GPT-5 Accelerates Scientific Discovery Across Multiple Fields  
OpenAI shares early research demonstrating how GPT-5 aids breakthroughs in math, physics, biology, and computer science by generating proofs and uncovering new 

### Put it together in a “main” flow
Tiny “orchestrator"

In [8]:
def run_daily_digest():
    print("Fetching articles...")
    articles = fetch_articles_from_rss(RSS_FEEDS)
    print(f"Got {len(articles)} articles.")

    print("Asking the model to summarise...")
    digest_markdown = make_news_digest_markdown(articles)

    print("\n=== DAILY AI DIGEST (Markdown) ===\n")
    print(digest_markdown)

    return digest_markdown

digest = run_daily_digest()


Fetching articles...
Got 20 articles.
Asking the model to summarise...

=== DAILY AI DIGEST (Markdown) ===

# Daily AI News Digest – 2025-11-24

### OpenAI and Foxconn Team Up to Boost U.S. AI Hardware Manufacturing  
OpenAI and Foxconn are partnering to design and produce next-generation AI infrastructure hardware domestically in the U.S. This collaboration aims to develop multiple generations of data-center systems, strengthen supply chains, and accelerate advanced AI infrastructure deployment.  
[Read more](https://openai.com/index/openai-and-foxconn-collaborate)

### OpenAI Launches Small Business AI Jam to Empower 1,000 Businesses  
OpenAI is collaborating with DoorDash, SCORE, and local groups to provide hands-on AI tools and training to 1,000 small businesses. The initiative helps Main Street entrepreneurs compete and grow by integrating AI into their operations.  
[Read more](https://openai.com/index/small-business-ai-jam)

### GPT-5 Accelerates Scientific Discovery Across Mult

In [9]:
with open("digest_preview.md", "w", encoding="utf-8") as f:
    f.write(digest)

print("Saved to digest_preview.md")

Saved to digest_preview.md


#### Send an email function

In [10]:
import smtplib
from email.mime.text import MIMEText
from email.utils import formatdate

# Load SMTP config from .env
SMTP_HOST = os.getenv("SMTP_HOST")
SMTP_PORT = int(os.getenv("SMTP_PORT", "587"))
SMTP_USER = os.getenv("SMTP_USER")
SMTP_PASS = os.getenv("SMTP_PASS")
FROM_EMAIL = os.getenv("FROM_EMAIL")
TO_EMAIL = os.getenv("TO_EMAIL")

print("Successfully loaded SMTP config")

 


Successfully loaded SMTP config


In [11]:
def send_email(subject, body_text):
    """
    Sends a plain-text email using SMTP.
    """
    if not all([SMTP_HOST, SMTP_PORT, SMTP_USER, SMTP_PASS, FROM_EMAIL, TO_EMAIL]):
        raise ValueError("Some SMTP settings are missing in your .env file")

    # Create email message
    msg = MIMEText(body_text, "plain", "utf-8")
    msg["Subject"] = subject
    msg["From"] = FROM_EMAIL
    msg["To"] = TO_EMAIL
    msg["Date"] = formatdate(localtime=True)

    # Send it
    with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
        server.starttls()            # Secure connection
        server.login(SMTP_USER, SMTP_PASS)
        server.send_message(msg)

    print("Email sent")

In [12]:
# Send the digest
# 
today_str = datetime.now().strftime("%Y-%m-%d")
subject = f"Daily AI Digest – {today_str}"

send_email(subject, digest)


Email sent


### AI agent orchestration
Instead of hardcoding the steps (fetch → summarise → email), an LLM-based agent decide which functions (“tools”) to call and in what order.

- Learn how to turn existing functions into tools
- Wrap them in a very simple agent loop   

Wrap functions as tools

In [13]:
import json

# --- 1. Wrap our existing functions as tools ---

def tool_fetch_news():
    """Fetch AI news articles from RSS feeds."""
    articles = fetch_articles_from_rss(RSS_FEEDS)
    return articles  # list of dicts

def tool_summarise_news(articles):
    """Summarise a list of articles into a Markdown digest."""
    digest = make_news_digest_markdown(articles)
    return digest

def tool_send_digest_email(subject, digest_markdown):
    """Send the digest via email."""
    send_email(subject, digest_markdown)
    return "Email sent"


In [14]:
# Dictionary of the tools the agent can "see" and call 


TOOLS = {
    "fetch_news": {
        "description": "Fetch AI news articles from predefined RSS feeds.",
        "function": tool_fetch_news,
    },
    "summarise_news": {
        "description": "Create a human-readable Markdown digest from a list of articles.",
        "function": tool_summarise_news,
    },
    "send_digest_email": {
        "description": "Send the given digest to the configured email address.",
        "function": tool_send_digest_email,
    },
}


#### Let the LLM decide which tool to call next
mini “agent loop”:
- give it a goal
- tell him what tools exist
 

In [15]:
def call_orchestrator(goal, state):
    """
    Ask the LLM what to do next.
    - goal: high-level goal string
    - state: dict with what we already have (articles, digest, etc.)
    Returns either:
      - {"type": "tool", "name": ..., "args": {...}}
      - {"type": "finish", "message": "..."}
    """
    # Prepare a description of tools
    tools_desc_lines = []
    for name, meta in TOOLS.items():
        tools_desc_lines.append(f"- {name}: {meta['description']}")
    tools_desc = "\n".join(tools_desc_lines)

    # Safely handle articles being None
    articles_in_state = state.get("articles") or []  # if None, use []

    # Summarise current state for the agent
    state_summary = {
        "has_articles": bool(articles_in_state),
        "num_articles": len(articles_in_state),
        "has_digest": bool(state.get("digest")),
        "email_already_sent": state.get("email_sent", False),
    }

    user_content = f"""
Your goal: {goal}

You have these tools available:
{tools_desc}

Current state (JSON):
{json.dumps(state_summary, indent=2)}

Rules:
- If you don't have any articles yet, call fetch_news.
- If you have articles but no digest, call summarise_news.
- If you have a digest and the email has not been sent yet, call send_digest_email.
- When everything is done, respond with FINISH and a brief message.

You must answer in one of these two JSON formats:

1) To call a tool:
{{
  "type": "tool",
  "name": "<tool-name>",
  "args": {{}}
}}

2) To finish:
{{
  "type": "finish",
  "message": "<what you accomplished>"
}}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are an orchestration agent that strictly follows the rules and JSON format."},
            {"role": "user", "content": user_content},
        ],
        temperature=0.0,
    )

    raw = response.choices[0].message.content
    try:
        decision = json.loads(raw)
    except json.JSONDecodeError:
        raise ValueError(f"Agent returned invalid JSON: {raw}")

    return decision


Make the agent run the workflow by itself

Gives the model the goal + Lists tools + describes the current state
Forces it to answer in pure JSON so we can parse it in Python (use the answer for later steps).

In [16]:
def run_news_agent():
    # The agent's goal
    goal = "Create today's AI news digest and email it to the user."

    # This is the state the agent keeps updating
    state = {
        "articles": None,
        "digest": None,
        "email_sent": False,
    }

    today_str = datetime.now().strftime("%Y-%m-%d")
    subject = f"Daily AI Digest – {today_str}"

    for step in range(10):  # safety limit: max 10 steps
        print(f"\n--- Agent step {step+1} ---")
        decision = call_orchestrator(goal, state)
        print("Agent decision:", decision)

        if decision.get("type") == "finish":
            print("\nAgent finished:", decision.get("message"))
            break

        if decision.get("type") == "tool":
            name = decision.get("name")
            if name not in TOOLS:
                raise ValueError(f"Agent asked for unknown tool: {name}")

            tool_fn = TOOLS[name]["function"]

            # Handle each tool explicitly (since they have different signatures)
            if name == "fetch_news":
                articles = tool_fn()
                state["articles"] = articles
                print(f"Tool fetch_news returned {len(articles)} articles.")

            elif name == "summarise_news":
                if not state.get("articles"):
                    print("No articles in state; skipping summarise.")
                else:
                    digest = tool_fn(state["articles"])
                    state["digest"] = digest
                    print("Tool summarise_news produced a digest.")

            elif name == "send_digest_email":
                if not state.get("digest"):
                    print("No digest in state; cannot send email.")
                else:
                    tool_fn(subject, state["digest"])
                    state["email_sent"] = True
                    print("Tool send_digest_email sent the email.")

        else:
            raise ValueError(f"Unexpected decision type: {decision}")

    return state


In [17]:
final_state = run_news_agent()


--- Agent step 1 ---
Agent decision: {'type': 'tool', 'name': 'fetch_news', 'args': {}}
Tool fetch_news returned 20 articles.

--- Agent step 2 ---
Agent decision: {'type': 'tool', 'name': 'summarise_news', 'args': {}}
Tool summarise_news produced a digest.

--- Agent step 3 ---
Agent decision: {'type': 'tool', 'name': 'send_digest_email', 'args': {}}
Email sent
Tool send_digest_email sent the email.

--- Agent step 4 ---
Agent decision: {'type': 'finish', 'message': "Today's AI news digest is ready and has already been emailed to you."}

Agent finished: Today's AI news digest is ready and has already been emailed to you.


### QA

In [18]:
print(tool_fetch_news())

[{'title': 'OpenAI and Foxconn collaborate to strengthen U.S. manufacturing across the AI supply chain', 'link': 'https://openai.com/index/openai-and-foxconn-collaborate', 'summary': 'OpenAI and Foxconn are collaborating to design and manufacture next-generation AI infrastructure hardware in the U.S. The partnership will develop multiple generations of data-center systems, strengthen U.S. supply chains, and build key components domestically to accelerate advanced AI infrastructure.', 'source': 'OpenAI News'}, {'title': 'Helping 1,000 small businesses build with AI', 'link': 'https://openai.com/index/small-business-ai-jam', 'summary': 'OpenAI is partnering with DoorDash, SCORE, and local organizations to help 1,000 small businesses build with AI. The Small Business AI Jam gives Main Street business owners hands-on tools and training to compete and grow.', 'source': 'OpenAI News'}, {'title': 'Early experiments in accelerating science with GPT-5', 'link': 'https://openai.com/index/acceler

In [19]:
articles_test = fetch_articles_from_rss(RSS_FEEDS)
print(type(articles_test), len(articles_test))

<class 'list'> 20


In [20]:
print("RSS_FEEDS =", RSS_FEEDS)
print("Fetch returns:", fetch_articles_from_rss(RSS_FEEDS))


RSS_FEEDS = ['https://openai.com/blog/rss.xml', 'https://www.theverge.com/rss/ai-artificial-intelligence/index.xml', 'https://feeds.feedburner.com/Techcrunch/artificial-intelligence']
Fetch returns: [{'title': 'OpenAI and Foxconn collaborate to strengthen U.S. manufacturing across the AI supply chain', 'link': 'https://openai.com/index/openai-and-foxconn-collaborate', 'summary': 'OpenAI and Foxconn are collaborating to design and manufacture next-generation AI infrastructure hardware in the U.S. The partnership will develop multiple generations of data-center systems, strengthen U.S. supply chains, and build key components domestically to accelerate advanced AI infrastructure.', 'source': 'OpenAI News'}, {'title': 'Helping 1,000 small businesses build with AI', 'link': 'https://openai.com/index/small-business-ai-jam', 'summary': 'OpenAI is partnering with DoorDash, SCORE, and local organizations to help 1,000 small businesses build with AI. The Small Business AI Jam gives Main Street b

#### This homemade version of the Agent/Orchestration worked well. 
#### Now , let's try to make it a bit more automated, using Langchain 

Conceptually:
- Keep existing functions:  fetch_articles_from_rss , make_news_digest_markdown , send_email

- Wrap these as LangChain tools
- Create a LangChain agent (LLM-powered) that:
- Has a goal: “Create today’s AI news digest and email it”
- Can call those tools in whatever order makes sense

Currently run from same notebook - could be neater in separate code? 

In [21]:
#pip install "langchain>=0.3.0" "langchain-openai>=0.2.0"

In [22]:
# Set up LangChain 

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate


c:\Users\brethm01\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (c:\Users\brethm01\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain\agents\__init__.py)

Wrap the functions as LangChain tools

In [ ]:
# 4.1 Fetch news tool 

@tool
def fetch_news() -> list:
    """Fetch AI news articles from predefined RSS feeds."""
    articles = fetch_articles_from_rss(RSS_FEEDS)
    # LangChain tools must return JSON-serializable data
    return articles


In [ ]:
# 4.2 Summarise news tool

@tool
def summarise_news(articles: list) -> str:
    """Summarise a list of AI news articles into a Markdown digest."""
    digest = make_news_digest_markdown(articles)
    return digest


In [ ]:
# 4.3 Send email tool

@tool
def send_digest_email(subject: str, digest_markdown: str) -> str:
    """Send the given digest via email using the configured SMTP settings."""
    send_email(subject, digest_markdown)
    return "Email sent successfully."


In [ ]:
# Tools list 
tools = [fetch_news, summarise_news, send_digest_email]

Create the LangChain LLM and agent

In [ ]:
# LangChain LLM wrapper (uses OPENAI_API_KEY from your env)
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.0,
)


define the prompt (agent instructions):

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """You are an AI assistant that creates a daily AI news digest and emails it to the user.

Your tools:
- fetch_news: get AI-related articles.
- summarise_news: turn articles into a human-readable Markdown digest.
- send_digest_email: send the digest via email.

Goal:
1. Fetch today's AI news.
2. Summarise it into a clear, concise Markdown digest.
3. Send the digest to the user via email.

Important:
- Always call fetch_news first if you don't have articles.
- After you have articles, call summarise_news.
- Once you have the digest, call send_digest_email with a suitable subject.
- When everything is complete, stop.

Think step by step and use tools when needed.
"""),
        ("human", "{input}"),
    ]
)


create the agent + executor:

In [ ]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


Run the LangChain agent
we ask the agent to do the whole job:

In [ ]:
today_str = datetime.now().strftime("%Y-%m-%d")
subject = f"Daily AI Digest – {today_str}"

result = agent_executor.invoke(
    {
        "input": f"Create today's AI news digest and email it to me. Use '{subject}' as the email subject."
    }
)
